# Preprocessing Code

This code includes the audio pre-processing as well as the feature extraction for the training data

## Code Setup

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from __future__ import division
from __future__ import print_function
from os import path
from pathlib import Path
import os, glob, torch,torchaudio, re
from python_speech_features import delta
from python_speech_features import mfcc

import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wav
import sys
import speech_dtw.qbe as qbe

from transformers import WavLMModel
from sklearn.decomposition import PCA

sys.path.append("..")
sys.path.append(path.join("..", "utils"))

/Users/camrynabrahamson/Documents/Engineering/Fourth Year/Skripsie/GitCoding/Evaluation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/camrynabrahamson/Documents/Engineering/Fourth Year/Skripsie/GitCoding/Evaluation/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pre-processing Function

In [11]:
# Removing silence and resampling recordings
SAMPLE_RATE = 16000 

# Load Silero VAD
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=False,
                              trust_repo=True)
(get_speech_timestamps, save_audio, read_audio, VADIterator, collect_chunks) = utils

def preProcessAudio(folder):
    for wav_fn in Path(folder).rglob("*.wav"): #Loop through all .wav files in folder and subfolders
        wav = read_audio(str(wav_fn), sampling_rate=SAMPLE_RATE)  # loads & resamples to 16kHz mono
        ts = get_speech_timestamps(
            wav, model, sampling_rate=SAMPLE_RATE,
            threshold=0.35,
            speech_pad_ms=300,
            min_speech_duration_ms=150,
            min_silence_duration_ms=300,
        ) #Gets which timestamps have speech
        if ts:  # If there is speech detected, save it
            wav_clean = collect_chunks(ts, wav)
            save_audio(str(wav_fn), wav_clean, sampling_rate=SAMPLE_RATE)  # overwrite
            print("Processed", wav_fn)

Using cache found in /Users/camrynabrahamson/.cache/torch/hub/snakers4_silero-vad_master


Example usage:

In [4]:
preProcessAudio("TestingData/OnlyNumbers/Afrikaans")

/Users/camrynabrahamson/.cache/torch/hub/snakers4_silero-vad_master/src/silero_vad/utils_vad.py:139: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  list_backends = torchaudio.list_audio_backends()
/Users/camrynabrahamson/.cache/torch/hub/snakers4_silero-vad_master/src/silero_vad/utils_vad.py:150: UserWarning: torchaudio.sox_effects.sox_effects.apply_effects_file has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  wav, sr = torcha

Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_05.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_04.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_06.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_07.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_03.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_02.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_01.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_09.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_08.wav
Processed TestingData/OnlyNumbers/Afrikaans/0boom/00_0.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_02.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_03.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_01.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_00.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_04.wav
Processed TestingData/OnlyNumbers/Afrikaans/4kwaad/04_05.wav
Processed TestingData/OnlyNumbers/A

In [12]:
preProcessAudio("TestingData/OnlyNumbers/English")

Processed TestingData/OnlyNumbers/English/C14/08_00.wav
Processed TestingData/OnlyNumbers/English/C14/06_00.wav
Processed TestingData/OnlyNumbers/English/C14/00_00.wav
Processed TestingData/OnlyNumbers/English/C14/05_00.wav
Processed TestingData/OnlyNumbers/English/C14/09_00.wav
Processed TestingData/OnlyNumbers/English/C14/07_00.wav
Processed TestingData/OnlyNumbers/English/C14/07_01.wav
Processed TestingData/OnlyNumbers/English/C14/01_00.wav
Processed TestingData/OnlyNumbers/English/C13/04_00.wav
Processed TestingData/OnlyNumbers/English/C13/06_00.wav
Processed TestingData/OnlyNumbers/English/C13/02_00.wav
Processed TestingData/OnlyNumbers/English/C13/00_00.wav
Processed TestingData/OnlyNumbers/English/C13/09_00.wav
Processed TestingData/OnlyNumbers/English/C13/07_00.wav
Processed TestingData/OnlyNumbers/English/C13/07_01.wav
Processed TestingData/OnlyNumbers/English/C13/03_00.wav
Processed TestingData/OnlyNumbers/English/C13/01_00.wav
Processed TestingData/OnlyNumbers/English/C12/04

## Feature Extraction

In [5]:
WAVLM_LAYER_INDEX = 6

# Select the WavLM model to be used here
device = "cpu"
model = WavLMModel.from_pretrained("microsoft/wavlm-base-plus").to(device).eval()

def cmvn(X):
    # X: [T, D] NumPy
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    return (X - mu) / (sd + 1e-8)

def getMFCCsFeatures(file): #A function which extracts MFCCs features from a given audio file
    sig, rate = torchaudio.load(file) #Reads the audio file, extracting the sample rate and signal data (as an array)

    #Check if sampled as correct sampling rate, if not - resample
    if rate != SAMPLE_RATE:
        print("Resampling", file ,"at 16kHz.\n")
        sig = torchaudio.functional.resample(sig, rate, SAMPLE_RATE)

    sig = sig.squeeze(0).numpy()

    #Extract features
    MFCC_static = mfcc(sig, SAMPLE_RATE) #Extracts MFCCs features given
    MFCC_deltas = delta(MFCC_static, 2) #Calculates delta (first derivative) of MFCCs features
    MFCC_delta_deltas = delta(MFCC_deltas, 2) #Calculates delta-delta (second derivative) of MFCCs features
    
    #Combine static, delta, and delta-delta features into a single feature vector
    features = np.hstack((MFCC_static, MFCC_deltas, MFCC_delta_deltas))
    features = cmvn(features) #Applies cepstral mean and variance normalization to features

    return features

def getWavLMFeatures(file): #A function which extracts MFCCs features from a given audio file
    sig, rate = torchaudio.load(file) #Reads the audio file, extracting the sample rate and signal data (as an array)

    #Check if sampled as correct sampling rate, if not - resample
    if rate != SAMPLE_RATE:
        print("Resampling", file ,"at 16kHz.\n")
        sig = torchaudio.functional.resample(sig, rate, SAMPLE_RATE)

    #Extracts layer 6 features
    sig = sig.to(device)
    with torch.inference_mode():
        out = model(sig, output_hidden_states=True)
        features = out.hidden_states[WAVLM_LAYER_INDEX].squeeze(0)  # [T, D] torch

    #Convert to numpy
    features = features.numpy()

    #Apply CMVN
    features = cmvn(features) #Applies cepstral mean and variance normalization to features
    
    return features

### Function to save the features as .pt files

In [6]:
def extractAndSaveFeatures(inputPath, outputPath, FeatureType): #Saves all features so they do not have to be re-extracted each time
    inputPath = Path(inputPath)
    outputPath  = Path(outputPath)

    #Expects naming to go A01/01_00.wav, where A01 is the speaker ID and 01 is the label
    #A02/01_02.wav would refer to adult speaker 2, where 01 is the label, and 02 is the utterance number (utterance 3)
    filePattern = re.compile(r"^\d{2}_\d{2}\.wav$")

    for wav_fn in inputPath.rglob("*.wav"):  # Loop through all .wav files
        fileName = wav_fn.stem                   # e.g. "03_01" or "noNum_01"
        prefix = fileName.split("_")[0]          # "03" or "noNum"

        # Handle label
        if prefix.isdigit():
            label = int(prefix)
        else:
            label = "No Number"  # e.g. "noNum_01.wav"  

        if FeatureType == "MFCCs":
            # Extract MFCC features
            features = getMFCCsFeatures(str(wav_fn)) #Gets MFCC features as numpy array
            features = torch.from_numpy(features).float() #Convert to torch tensor
        elif FeatureType == "WavLM":
            # Extract WavLM features
            features = getWavLMFeatures(str(wav_fn)) #Gets WavLM features as numpy array
            features = torch.from_numpy(features).float() #Convert to torch tensor

        # Build save path
        rel_path = wav_fn.relative_to(inputPath)   # e.g. A01/03_00.wav
        save_path = outputPath / rel_path.with_suffix(".pt")
        save_path.parent.mkdir(parents=True, exist_ok=True)

        # Package data
        data = {
            "features": features,
            "label": label,
            "speaker": wav_fn.parent.name,
            "utt_id": fileName,
            "feat_type": FeatureType
        }

        torch.save(data, save_path)
        print(f"Saved {save_path}")

    print("Feature extraction complete.")


Example usage:

In [7]:
extractAndSaveFeatures("TrainingData/Child/Afrikaans", "TrainingFeatures/WavLMBase+/Afrikaans", "WavLM")

Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_05.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_04.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_06.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_07.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_03.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_02.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_00.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_01.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_09.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/0boom/00_08.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_02.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_03.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_01.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_00.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_04.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad/04_05.pt
Saved TrainingFeatures/WavLMBase+/Afrikaans/4kwaad